<font>
<div dir=ltr align=center>
<img src='Sharif_logo.png' width=250 height=250> <br>
<font color=0F5298 size=7>
Applied Data Science<br>
<font color=2565AE size=5>
Spring 2025<br>
<font color=3C99D size=5>
HW6 - Regression Methods <br>
<font color=696880 size=4>
Ali Mohammadzade Shabestari - 401106482 - Computer Engineering



# 1. Import Libraries

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


# 2. Load & Split Dataset

In [85]:
data = pd.read_excel('Concrete_Data.xls')

X, y = data.iloc[:, :-1].values, data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize X
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Regression Tasks

## 3. 1. Linear Regression

### 3. 1. 1. R2 Score > 0.8

In [86]:
# Lasso Regression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("Linear Regression R2 Score:", r2)

Linear Regression R2 Score: 0.6275416055429022


To achieve a better R² score, we can use polynomial features to capture the non-linear relationships in the data. By transforming the input features into higher-degree polynomial terms, the model can better fit the underlying patterns in the data.

In [87]:
# Create polynomial features of degree 2
X_poly2 = PolynomialFeatures(degree=2)
X_train_poly2 = X_poly2.fit_transform(X_train)
X_test_poly2 = X_poly2.transform(X_test)
scaler = StandardScaler()
X_train_poly2 = scaler.fit_transform(X_train_poly2)
X_test_poly2 = scaler.transform(X_test_poly2)

# Linear Regression with Polynomial Features
poly_regressor = LinearRegression()
poly_regressor.fit(X_train_poly2, y_train)
y_pred = poly_regressor.predict(X_test_poly2)
r2_poly = r2_score(y_test, y_pred)
print("Polynomial Regression (degree = 2) R2 Score:", r2_poly)

Polynomial Regression (degree = 2) R2 Score: 0.784268504972648


### 3. 1. 2. R2 Score > 0.85

We can create polynomial features of dgree 3 or higher, to get better results.

In [88]:
X_poly3 = PolynomialFeatures(degree=3)
X_train_poly3 = X_poly3.fit_transform(X_train)
X_test_poly3 = X_poly3.transform(X_test)
scaler = StandardScaler()
X_train_poly3 = scaler.fit_transform(X_train_poly3)
X_test_poly3 = scaler.transform(X_test_poly3)
# Linear Regression with Polynomial Features
poly_regressor = LinearRegression()
poly_regressor.fit(X_train_poly3, y_train)
y_pred = poly_regressor.predict(X_test_poly3)
r2_poly = r2_score(y_test, y_pred)
print("Polynomial Regression (degree = 3)  R2 Score:", r2_poly)

Polynomial Regression (degree = 3)  R2 Score: 0.8437173906891395


## 3. 2. Kernel Regression

Kernel regression is a non-parametric technique used to model complex relationships between variables. It leverages kernel functions to map input data into a higher-dimensional space, enabling the model to capture non-linear patterns.

 Common kernel functions include Radial Basis Function (RBF), polynomial, and linear kernels. In Python, `KernelRidge` from `sklearn.kernel_ridge` can be used for kernel regression. Key parameters include `alpha` (regularization strength) and `kernel` (type of kernel). The model is trained using the `fit()` method and predictions are made using the `predict()` method.

In [89]:
# Perform Kernel Regression
kernel_ridge = KernelRidge(kernel='rbf', alpha=0.5)
kernel_ridge.fit(X_train_scaled, y_train)
y_pred_kernel_ridge = kernel_ridge.predict(X_test_scaled)

# Caculate Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred_kernel_ridge)
print("MAE:", mae)

MAE: 4.897926237984521


## 3. 3. Logistic Regression

In [90]:
# First generate the categorical labels
y_train_labels = pd.qcut(y_train, q=2, labels=['low', 'high'])
y_test_labels = pd.qcut(y_test, q=2, labels=['low', 'high'])

# Then encode them
label_encoder = LabelEncoder()
label_encoder.fit(['low', 'high'])  # or just fit on y_train_labels
y_train_multi = label_encoder.transform(y_train_labels)
y_test_multi = label_encoder.transform(y_test_labels)

# Perform Logistic Regression
logistic_regressor = LogisticRegression(solver='lbfgs', max_iter=1000)
logistic_regressor.fit(X_train_scaled, y_train_multi)
y_pred_multi = logistic_regressor.predict(X_test_scaled)

# Calculate r2 score
r2_logistic = r2_score(y_test_multi, y_pred_multi)
print("Logistic Regression R2 Score:", r2_logistic)

Logistic Regression R2 Score: 0.5145631067961165


Umm, let's perform it on Polynomial Features (degree = 5). We expect to get better results.

In [92]:
# Perform Logistic Regression
logistic_regressor = LogisticRegression(solver='lbfgs', max_iter=5000)
logistic_regressor.fit(X_train_poly3, y_train_multi)
y_pred_multi = logistic_regressor.predict(X_test_poly3)

# Calculate r2 score
r2_logistic = r2_score(y_test_multi, y_pred_multi)
print("Logistic Regression R2 Score:", r2_logistic)

# Calculate Accuracy
accuracy = accuracy_score(y_test_multi, y_pred_multi)
print("Logistic Regression Accuracy:", accuracy)

Logistic Regression R2 Score: 0.5339805825242718
Logistic Regression Accuracy: 0.883495145631068


Looks like the improvment is good but not enough. Itried higher degrees, but they didn't make a big effect. (Maximum R2 was observed at degree = 5)

I guess since the structure of data is not designed for a classification problem, reaching a higher R2 Score is not a must. Just look at accuracy!

## 3. 4. Ridge Regression

The Ridge Regression minimizes the following cost function:

$$
J(\mathbf{w}) = \sum_{i=1}^{n} (y_i - \mathbf{w}^T \mathbf{x}_i)^2 + \alpha \|\mathbf{w}\|^2_2
$$

where 
- `w` is the weight vector (model parameters)  
- `xᵢ` is the input features for instance *i*  
- `yᵢ` is the true value for instance *i*  
- `α ≥ 0` is the regularization strength (hyperparameter)  
- `‖w‖²₂ = ∑ wⱼ²` is the L2 norm (sum of squared coefficients)


In [ ]:
# Perform Ridge Regression with new parameters
ridge_regressor = Ridge(alpha=0.5)  
ridge_regressor.fit(X_train, y_train)
y_pred_ridge = ridge_regressor.predict(X_test)

# Calculate R2 Score
r2_ridge = r2_score(y_test, y_pred_ridge)
print("Ridge Regression R2 Score:", r2_ridge)

Ridge Regression R2 Score: 0.6275416891423036


Let's try Polynomial Features of degree = 3.

In [97]:
ridge_regressor.fit(X_train_poly3, y_train)
y_pred_ridge = ridge_regressor.predict(X_test_poly3)

# Calculate R2 Score
r2_ridge = r2_score(y_test, y_pred_ridge)
print("Ridge Regression R2 Score:", r2_ridge)

Ridge Regression R2 Score: 0.8411685341628778


## 3. 5. Lasso Regression

Lasso Regression minimizes the following cost function:

$$
J(\mathbf{w}) = \sum_{i=1}^{n} (y_i - \mathbf{w}^T \mathbf{x}_i)^2 + \alpha \|\mathbf{w}\|_1
$$

**Where:**

- `w` is the weight vector (model parameters)  
- `xᵢ` is the input features for instance *i*  
- `yᵢ` is the true value for instance *i*  
- `α ≥ 0` is the regularization strength (hyperparameter)  
- `‖w‖₁ = ∑ |wⱼ|` is the L1 norm (sum of absolute values of coefficients)

In [104]:
# Perform Lasso Regression
lasso_regressor = Lasso(alpha=0.3)  # You can adjust the alpha parameter
lasso_regressor.fit(X_train, y_train)
y_pred_lasso = lasso_regressor.predict(X_test)

# Calculate R2 Score
r2_lasso = r2_score(y_test, y_pred_lasso)
print("Lasso Regression R2 Score:", r2_lasso)

Lasso Regression R2 Score: 0.627594030943751


Let's try Polynomial Features of degree = 3.

In [106]:
lasso_regressor.fit(X_train_poly3, y_train)
y_pred_lasso = lasso_regressor.predict(X_test_poly3)

# Calculate R2 Score
r2_lasso = r2_score(y_test, y_pred_lasso)
print("Lasso Regression R2 Score:", r2_lasso)

Lasso Regression R2 Score: 0.7884916793037768


## 3. 6. How Kernel method helps?

The kernel trick is a method used in machine learning (especially in models like Support Vector Machines and kernelized regression) to implicitly map input data into a higher-dimensional feature space without actually computing the transformation. Instead, it uses a kernel function (like RBF or polynomial) that calculates the inner product of data points in that higher-dimensional space directly.

In standard linear regression, if the data is not linearly related to the target, performance suffers. But by using the kernel trick in Kernel Ridge Regression or SVR (Support Vector Regression), we can capture complex, nonlinear relationships in the data—leading to much better fits and predictions without explicitly increasing the model complexity.